In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.schema import Document

In [2]:
embeddings = HuggingFaceEmbeddings(
                model_name="BAAI/bge-small-en-v1.5",
                model_kwargs={"device": "cpu"},
                encode_kwargs={"normalize_embeddings": True}
            )

In [3]:
# Create LangChain documents for IPL players
doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [4]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [5]:
vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory="chroma_db",
    collection_name="ipl_players"
)

In [6]:
vector_store.add_documents(docs)

['ce59c5f2-2f19-401f-a449-acab678fcf20',
 'b87b0c6e-0ff4-4a77-87b1-1c8a12ad26a9',
 '7ef109b0-32f5-4346-aeca-38f40d69d788',
 '292a353f-d246-417d-971f-c8263e5fb758',
 '6c87c119-d1c5-47e5-a900-5fc5ddab6ebd']

In [7]:
vector_store.get(include=['embeddings', 'metadatas', 'documents'])

{'ids': ['ce59c5f2-2f19-401f-a449-acab678fcf20',
  'b87b0c6e-0ff4-4a77-87b1-1c8a12ad26a9',
  '7ef109b0-32f5-4346-aeca-38f40d69d788',
  '292a353f-d246-417d-971f-c8263e5fb758',
  '6c87c119-d1c5-47e5-a900-5fc5ddab6ebd'],
 'embeddings': array([[-0.03472489,  0.0447316 , -0.05392007, ..., -0.01590654,
          0.04077549,  0.01395198],
        [-0.00889341,  0.02867057,  0.00030774, ...,  0.00095008,
          0.04364204,  0.00328573],
        [-0.0446614 ,  0.07488782,  0.00069516, ..., -0.03354059,
          0.04045827,  0.01402278],
        [-0.0042139 ,  0.0372008 , -0.01846489, ...,  0.0277884 ,
          0.09400118,  0.02098929],
        [ 0.0111636 ,  0.05875369, -0.03539582, ...,  0.02071979,
          0.02770315, -0.00161607]], shape=(5, 384)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the mo

In [8]:
vector_store.similarity_search("Who amon these are bowler", k=2)

[Document(id='292a353f-d246-417d-971f-c8263e5fb758', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(id='b87b0c6e-0ff4-4a77-87b1-1c8a12ad26a9', metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.")]

In [9]:
vector_store.similarity_search_with_score("Who amon these are bowler", k=2)

[(Document(id='292a353f-d246-417d-971f-c8263e5fb758', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.6227759718894958),
 (Document(id='b87b0c6e-0ff4-4a77-87b1-1c8a12ad26a9', metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure."),
  0.7512608170509338)]

In [11]:
vector_store.similarity_search_with_score(
    query="",
    filter={"team": "Mumbai Indians"},
)

[(Document(id='b87b0c6e-0ff4-4a77-87b1-1c8a12ad26a9', metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure."),
  1.0175776481628418),
 (Document(id='292a353f-d246-417d-971f-c8263e5fb758', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  1.0492128133773804)]

In [12]:
# update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(
    document_id="ce59c5f2-2f19-401f-a449-acab678fcf20",
    document=updated_doc1
)

In [13]:
vector_store.get(include=['embeddings', 'metadatas', 'documents'])

{'ids': ['ce59c5f2-2f19-401f-a449-acab678fcf20',
  'b87b0c6e-0ff4-4a77-87b1-1c8a12ad26a9',
  '7ef109b0-32f5-4346-aeca-38f40d69d788',
  '292a353f-d246-417d-971f-c8263e5fb758',
  '6c87c119-d1c5-47e5-a900-5fc5ddab6ebd'],
 'embeddings': array([[-0.00227819,  0.06583228, -0.04027405, ..., -0.02917498,
          0.02682617,  0.03997119],
        [-0.00889341,  0.02867057,  0.00030774, ...,  0.00095008,
          0.04364204,  0.00328573],
        [-0.0446614 ,  0.07488782,  0.00069516, ..., -0.03354059,
          0.04045827,  0.01402278],
        [-0.0042139 ,  0.0372008 , -0.01846489, ...,  0.0277884 ,
          0.09400118,  0.02098929],
        [ 0.0111636 ,  0.05875369, -0.03539582, ...,  0.02071979,
          0.02770315, -0.00161607]], shape=(5, 384)),
 'documents': ["Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple ce

In [14]:
vector_store.delete(ids=['ce59c5f2-2f19-401f-a449-acab678fcf20'])

In [15]:
vector_store.get(include=['embeddings', 'metadatas', 'documents'])

{'ids': ['b87b0c6e-0ff4-4a77-87b1-1c8a12ad26a9',
  '7ef109b0-32f5-4346-aeca-38f40d69d788',
  '292a353f-d246-417d-971f-c8263e5fb758',
  '6c87c119-d1c5-47e5-a900-5fc5ddab6ebd'],
 'embeddings': array([[-0.00889341,  0.02867057,  0.00030774, ...,  0.00095008,
          0.04364204,  0.00328573],
        [-0.0446614 ,  0.07488782,  0.00069516, ..., -0.03354059,
          0.04045827,  0.01402278],
        [-0.0042139 ,  0.0372008 , -0.01846489, ...,  0.0277884 ,
          0.09400118,  0.02098929],
        [ 0.0111636 ,  0.05875369, -0.03539582, ...,  0.02071979,
          0.02770315, -0.00161607]], shape=(4, 384)),
 'documents': ["Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
  'MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.',
  'Jasprit Bumrah is